In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import RobustScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.ensemble import RandomForestClassifier

df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')



In [64]:
from sklearn.preprocessing import LabelEncoder


# Определяем категориальные признаки
object_columns = df_train.select_dtypes(include=['object']).columns

# Инициализируем LabelEncoder
label_encoder = LabelEncoder()

# Конвертируем данные типа object  в численные значения:
for col in object_columns:
    df_train[col] = label_encoder.fit_transform(df_train[col])

df_train.head(5).style.set_properties(**{"background-color": "#A8DADC", "color": "black", "border": "1.5px solid White"})

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,0,21532,1,0,0,0,0,2.300000,316.000000,3.350000,172.000000,1601.000000,179.800000,63.000000,394.000000,9.700000,3.000000,2
1,1,2574,1,19237,0,0,0,0,0,0.900000,364.000000,3.540000,63.000000,1440.000000,134.850000,88.000000,361.000000,11.000000,3.000000,0
2,2,3428,1,13727,0,0,1,1,2,3.300000,299.000000,3.550000,131.000000,1029.000000,119.350000,50.000000,199.000000,11.700000,4.000000,2
3,3,2576,1,18460,0,0,0,0,0,0.600000,256.000000,3.500000,58.000000,1653.000000,71.300000,96.000000,269.000000,10.700000,3.000000,0
4,4,788,1,16658,0,0,1,0,0,1.100000,346.000000,3.650000,63.000000,1181.000000,125.550000,96.000000,298.000000,10.600000,4.000000,0


In [68]:
X = df_train.drop(['Status'], axis=1).select_dtypes(include=[np.number])  # Используем только числовые
y = df_train['Status'].values

count_y_equals_1 = np.sum(y == 1)
count_y_equals_0 = np.sum(y == 0)
count_y_equals_2 = np.sum(y == 2)

# Вывод количества значений y = 1
print(f'Количество значений y = 1: {count_y_equals_1}')
print(f'Количество значений y = 0: {count_y_equals_0}')
print(f'Количество значений y = 2: {count_y_equals_2}')

# Нормализация данных
X = (X - X.mean()) / X.std()

# Разделение данных на обучающую и тестовую выборки
train_size = int(0.7 * len(X))  # 70% на обучение
X_train, X_test = X.values[:train_size], X.values[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


# Определение кастомного класса для логистической регрессии, который содержит методы
# для обучения (fit), предсказания (predict) и оценки (evaluate).
# Методы класса:
# _softmax: вычисляет для входных данных вероятности классов.
# fit: выполняет обучение модели, используя градиентный спуск.
# predict: возвращает предсказанные классы на основе входящих данных.
# evaluate: вычисляет метрики, такие как точность, полнота и F1-score для каждой категории.
class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def _softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Для численной стабильности
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        self.weights = np.zeros((num_features, num_classes))
        self.bias = np.zeros(num_classes)

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._softmax(linear_model)

            # Преобразуем y в one-hot представление
            y_one_hot = np.eye(num_classes)[y]

            # Градиентный спуск
            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y_one_hot))
            db = (1 / num_samples) * np.sum(y_predicted - y_one_hot, axis=0)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._softmax(linear_model)
        return np.argmax(y_predicted, axis=1)  # Получаем индексы классов с наибольшей вероятностью

    def evaluate(self, y_true, y_pred):
        metrics = {}
        unique_classes = np.unique(y_true)

        for label in unique_classes:
            TP = np.sum((y_pred == label) & (y_true == label))
            TN = np.sum((y_pred != label) & (y_true != label))
            FP = np.sum((y_pred == label) & (y_true != label))
            FN = np.sum((y_pred != label) & (y_true == label))

            accuracy = (TP + TN) / len(y_true)
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            metrics[label] = {
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1
            }

        return metrics

# Создаем модель логистической регрессии
model = LogisticRegressionCustom()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
metrics = model.evaluate(y_test, predictions)

# Вывод метрик для каждого значения Status
for label, metric_values in metrics.items():
    print(f'Class {label}:')
    print('Accuracy:', metric_values['Accuracy'])
    print('Precision:', metric_values['Precision'])
    print('Recall:', metric_values['Recall'])
    print('F1 Score:', metric_values['F1 Score'])
    print('---')

Количество значений y = 1: 275
Количество значений y = 0: 4965
Количество значений y = 2: 2665
Class 0:
Accuracy: 0.8145025295109612
Precision: 0.8259236067626801
Recall: 0.8906144496961512
F1 Score: 0.857050032488629
---
Class 1:
Accuracy: 0.9607925801011804
Precision: 1.0
Recall: 0.010638297872340425
F1 Score: 0.021052631578947368
---
Class 2:
Accuracy: 0.822512647554806
Precision: 0.7428940568475452
Recall: 0.7214554579673776
F1 Score: 0.732017823042648
---


## Посмотрим, какие признаки вносят наибольший вклад в обучение модели и после этого в датасете оставим только лучшие признаки

In [26]:
# import numpy as np
# import pandas as pd

# # Предполагается, что df_train уже существует и содержит необходимые данные.

# # Добавление новых признаков
# df_train['Liver_function'] = df_train['Bilirubin'] / df_train['Albumin']
# df_train['Inflammation'] = df_train['Alk_Phos'] / df_train['SGOT']
# df_train['Normal_Platelets'] = df_train['Platelets'] / df_train['Stage']
# df_train['Normal_Albumin'] = df_train['Albumin'] / (1 + df_train['Edema'])
# df_train['Liver_damage'] = df_train['Ascites'] + df_train['Hepatomegaly'] + df_train['Spiders']

# # Категоризация возраста
# def categorize_age_in_days(age_days):
#     if age_days <= 10950:
#         return 0  # Young
#     elif age_days <= 18250:
#         return 1  # Middle_Aged
#     elif age_days <= 25550:
#         return 2  # Senior
#     else:
#         return 3  # Elderly

# df_train['Age_Group'] = df_train['Age'].apply(categorize_age_in_days)

# df_train['High_Risk'] = ((df_train['Bilirubin'] > 2) &
#                          (df_train['Copper'] > 140) &
#                          (df_train['Alk_Phos'] > 150)).astype(int)

# df_train['ALBI'] = (np.log10(df_train['Bilirubin']) * 0.66) - (df_train['Albumin'] * 0.085)

# # Подготовка данных
# X = df_train.drop(['Status'], axis=1).select_dtypes(include=[np.number])  # Используем только числовые
# y = df_train['Status'].values

# # Нормализация данных
# X = (X - X.mean()) / X.std()

# # Разделение данных на обучающую и тестовую выборки
# train_size = int(0.8 * len(X))  # 80% на обучение
# X_train, X_test = X.values[:train_size], X.values[train_size:]
# y_train, y_test = y[:train_size], y[train_size:]

# class LogisticRegressionCustom:
#     def __init__(self, learning_rate=0.01, num_iterations=1000):
#         self.learning_rate = learning_rate
#         self.num_iterations = num_iterations
#         self.weights = None
#         self.bias = None

#     def _sigmoid(self, z):
#         return 1 / (1 + np.exp(-z))

#     def fit(self, X, y):
#         num_samples, num_features = X.shape
#         self.weights = np.zeros(num_features)
#         self.bias = 0

#         for _ in range(self.num_iterations):
#             linear_model = np.dot(X, self.weights) + self.bias
#             y_predicted = self._sigmoid(linear_model)

#             # Градиентный спуск
#             dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
#             db = (1 / num_samples) * np.sum(y_predicted - y)

#             self.weights -= self.learning_rate * dw
#             self.bias -= self.learning_rate * db

#     def predict(self, X):
#         linear_model = np.dot(X, self.weights) + self.bias
#         y_predicted = self._sigmoid(linear_model)
#         y_predicted_class = [1 if i > 0.5 else 0 for i in y_predicted]
#         return np.array(y_predicted_class)

#     def evaluate(self, y_true, y_pred):
#         TP = np.sum((y_pred == 1) & (y_true == 1))
#         TN = np.sum((y_pred == 0) & (y_true == 0))
#         FP = np.sum((y_pred == 1) & (y_true == 0))
#         FN = np.sum((y_pred == 0) & (y_true == 1))

#         accuracy = (TP + TN) / len(y_true)
#         precision = TP / (TP + FP) if (TP + FP) > 0 else 0
#         recall = TP / (TP + FN) if (TP + FN) > 0 else 0
#         f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#         return accuracy, precision, recall, f1

# # Создаем модель
# model = LogisticRegressionCustom()
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
# metrics = model.evaluate(y_test, predictions)

# print('Accuracy:', metrics[0])
# print('Precision:', metrics[1])
# print('Recall:', metrics[2])
# print('F1 Score:', metrics[3])

# # Вычисление влияния признаков (вывод всех признаков)
# feature_importance = np.abs(model.weights)
# feature_importance_series = pd.Series(feature_importance, index=X.columns)

# # Выводить все признаки с их весами
# print("Все признаки с весами значимости:")
# print(feature_importance_series)





Accuracy: 0.523719165085389
Precision: 0.14678899082568808
Recall: 0.6153846153846154
F1 Score: 0.23703703703703705
Все признаки с весами значимости:
id                  0.823801
N_Days              1.242137
Drug                0.194111
Age                 0.713788
Sex                 0.515358
Ascites             1.285286
Hepatomegaly        1.001728
Spiders             0.905116
Edema               1.618018
Bilirubin           1.794821
Cholesterol         0.568670
Albumin             0.867511
Copper              1.473864
Alk_Phos            0.590319
SGOT                1.164327
Tryglicerides       0.778061
Platelets           0.520004
Prothrombin         1.764685
Stage               1.126129
Liver_function      1.812096
Inflammation        0.185350
Normal_Platelets    0.827217
Normal_Albumin      1.559324
Liver_damage        1.391704
Age_Group           0.517757
High_Risk           1.524044
ALBI                2.106891
dtype: float64


По весам видно, что как и в предыдущей работе новые признаки имеют существенный вес, даже по сравнению со стандартными, но лучшие признаки отличаются по составу!

In [56]:
df_train['Liver_function'] = df_train['Bilirubin'] / df_train['Albumin']
df_train['Normal_Albumin'] = df_train['Albumin'] / (1 + df_train['Edema'])
df_train['Liver_damage'] = df_train['Ascites'] + df_train['Hepatomegaly'] + df_train['Spiders']

df_train['High_Risk'] = ((df_train['Bilirubin'] > 2) &
                         (df_train['Copper'] > 140) &
                         (df_train['Alk_Phos'] > 150)).astype(int)

df_train['ALBI'] = (np.log10(df_train['Bilirubin']) * 0.66) - (df_train['Albumin'] * 0.085)

# Сохраняем только нужные признаки
desired_columns = [
    'N_Days', 'Ascites', 'Edema', 'Liver_function', 'ALBI',
    'Bilirubin', 'Prothrombin', 'Copper',
    'Normal_Albumin', 'Age', 'SGOT', 'High_Risk'
]
df_train_filtered = df_train[desired_columns + ['Status']]

# Разделим данные на признаки и целевую переменную
X1 = df_train_filtered.drop(['Status'], axis=1)
y1 = df_train_filtered['Status']

# Нормализация данных
X1 = (X1 - X1.mean()) / X1.std()

# Разделение данных на обучающую и тестовую выборки
train_size = int(0.7 * len(X))  # 70% на обучение
X1_train, X1_test = X1.values[:train_size], X1.values[train_size:]
y1_train, y1_test = y1[:train_size], y1[train_size:]

class LogisticRegressionCustom1:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def _softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Для численной стабильности
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def fit(self, X, y):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        self.weights = np.zeros((num_features, num_classes))
        self.bias = np.zeros(num_classes)

        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._softmax(linear_model)

            # Преобразуем y в one-hot представление
            y_one_hot = np.eye(num_classes)[y]

            # Градиентный спуск
            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y_one_hot))
            db = (1 / num_samples) * np.sum(y_predicted - y_one_hot, axis=0)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._softmax(linear_model)
        return np.argmax(y_predicted, axis=1)  # Получаем индексы классов с наибольшей вероятностью

    def evaluate(self, y_true, y_pred):
        metrics = {}
        unique_classes = np.unique(y_true)

        for label in unique_classes:
            TP = np.sum((y_pred == label) & (y_true == label))
            TN = np.sum((y_pred != label) & (y_true != label))
            FP = np.sum((y_pred == label) & (y_true != label))
            FN = np.sum((y_pred != label) & (y_true == label))

            accuracy = (TP + TN) / len(y_true)
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            metrics[label] = {
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1
            }

        return metrics

# Создаем модель
model1 = LogisticRegressionCustom1()
model1.fit(X1_train, y1_train)
predictions1 = model1.predict(X1_test)
metrics1 = model1.evaluate(y1_test, predictions1)

# Вывод метрик для каждого значения Status
for label, metric_values in metrics.items():
    print(f'Class {label}:')
    print('Accuracy:', metric_values['Accuracy'])
    print('Precision:', metric_values['Precision'])
    print('Recall:', metric_values['Recall'])
    print('F1 Score:', metric_values['F1 Score'])
    print('---')


Class 0:
Accuracy: 0.8174536256323778
Precision: 0.8283208020050126
Recall: 0.8926401080351114
F1 Score: 0.8592785180370491
---
Class 1:
Accuracy: 0.9607925801011804
Precision: 0.6666666666666666
Recall: 0.02127659574468085
F1 Score: 0.041237113402061855
---
Class 2:
Accuracy: 0.8204047217537943
Precision: 0.7399741267787839
Recall: 0.7176913425345044
F1 Score: 0.7286624203821657
---


По сравнению с датасетом без новых признаков метрики незначительно увеличились, но по классу 1 Precision (доля правильно предсказанных положительных результатов (TP) к общему количеству предсказанных положительных результатов (TP + FP)) упала с 1 до 0,67